# Find false labels with Cleanlab

We use the [Cleanlab library](https://github.com/cleanlab/cleanlab) to compute label error scores. We then manually inspect the data points to correct them.

More information about this play can be found in the Spotlight documentation: [Find label errors with Cleanlab](https://renumics.com/docs/playbook/label-errors-cleanlab)

For more data-centric AI workflows, check out our [Awesome Open Data-centric AI](https://github.com/Renumics/awesome-open-data-centric-ai) list on Github.

## tldr

In [ ]:
#@title Install required packages with PIP

!pip install renumics-spotlight cleanlab datasets

In [ ]:
#@title Play as copy-n-paste functions
import datasets
from renumics import spotlight
from cleanlab.filter import find_label_issues
import numpy as np
import pandas as pd
import requests
import json


def label_error_score_cleanlab(df, probabilities_name='probabilities', label_name='labels'):
 
    probs = np.stack(df[probabilities_name].to_numpy())
    labels = df[label_name].to_numpy()
 
    label_issues = find_label_issues(labels, probs)
    
    df_out=pd.DataFrame()
    df_out['label_error_score']=label_issues    
   
    return df_out

## Step-by-step example on CIFAR-100

### Load CIFAR-100 from Huggingface hub and convert it to Pandas dataframe

In [ ]:
dataset = datasets.load_dataset("renumics/cifar100-enriched", split="all")

df = dataset.to_pandas()

### Compute label error scores with Cleanlab

In [ ]:
df_le = label_error_score_cleanlab(df, label_name='fine_label')
df = pd.concat([df, df_le], axis=1)

### Inspect label errors and remove them with Spotlight

> ⚠️ Running Spotlight in Colab currently has severe limitations (slow, no similarity map, no layouts) due to Colab restrictions (e.g. no websocket support). Run the notebook locally for the full Spotlight experience.

In [ ]:
df_show = df.drop(columns=['embedding', 'probabilities'])


# handle google colab differently
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    #visualization in Google Colab only works in chrome and does not support websockets, we need some hacks to visualize something
    df_show=df_show[:10000]
    df_show['embx'] =  [emb[0] for emb in df_show['embedding_reduced'] ]
    df_show['emby'] =  [emb[1] for emb in df_show['embedding_reduced'] ]
    port=50123
    layout_url="https://raw.githubusercontent.com/Renumics/spotlight/main/playbook/veteran/label_errors_cleanlab_colab.json"
    response = requests.get(layout_url)
    layout = spotlight.layout.nodes.Layout(**json.loads(response.text))
    spotlight.show(df_show, port=port, dtype={"image": spotlight.Image}, layout=layout)  
    from google.colab.output import eval_js  # type: ignore
    print(str(eval_js(f"google.colab.kernel.proxyPort({port}, {{'cache': true}})")))

else:
    df_show = df.drop(columns=['embedding', 'probabilities']) 
    layout_url="https://raw.githubusercontent.com/Renumics/spotlight/main/playbook/veteran/label_errors_cleanlab.json"
    response = requests.get(layout_url)
    layout = spotlight.layout.nodes.Layout(**json.loads(response.text))
    spotlight.show(df_show, dtype={"image": spotlight.Image, "embedding_reduced": spotlight.Embedding}, layout=layout)